# Check the performance of the agents
## Start the Environment

In [ ]:
# import sys
# !{sys.executable} -m pip install mlagents==0.22.0 gym_unity==0.22 

In [ ]:
import logging
import time
from collections import deque

from PIL import Image
import itertools
import gym
import numpy as np
import time
from collections import deque
from gym import error, spaces
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.registry import UnityEnvRegistry
from mlagents_envs.side_channel.environment_parameters_channel import (
    EnvironmentParametersChannel,
)
from mlagents_envs.side_channel.engine_configuration_channel import (
    EngineConfigurationChannel,
)
from gym_unity.envs import UnityToGymWrapper
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

from double_dqn.double_dqn_agent import Agent
from double_dqn.q_network import QNetwork



reset_parameters = EnvironmentParametersChannel()
engine_config = EngineConfigurationChannel()

environment_filename='/Users/ankit/Desktop/MTP2.0/Executables/3.app'

unity_env = UnityEnvironment(
        environment_filename,
        1,
        timeout_wait=30,
        side_channels=[reset_parameters, engine_config],
        )   

env = UnityToGymWrapper(unity_env, flatten_branched=True)

In [ ]:
env.reset()

In [ ]:
def train(n_episodes=1, max_t=3000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    done_count = 0
    file=None
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset()
        state = env_info 
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, info = env.step(action)        # send the action to the environment
            agent.step(state, action, reward, next_state, done)
            state = next_state
            print(reward)
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        file = open('scores.txt', 'a')
        file.write(str(i_episode)+','+ str(score)+","+str(eps)+"\n")
        file.close()
        if i_episode % 20 == 0:
            torch.save(agent.qnetwork_local.state_dict(), 'checkPoint.pth')
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\tMax Score: {:.2f}'.format(i_episode, np.mean(scores_window), np.max(scores_window)))
        if np.mean(scores_window)>=40.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint_dueling_ddqn.pth')
            break
    return scores

In [ ]:
agent = Agent(state_size=83, action_size=4, seed=0, QNetwork=QNetwork)
scores = train()

# For Inference

In [ ]:

def run(agent):
    env_info = env.reset()
    state = env_info            
    score = 0                                          # initialize the score
    for i in range(1000):
        action = agent.act(state, 0.2)                      # select an action
        print(action)
        next_state, reward, done, info = env.step(action)
        score += reward                                # update the score
#         print(state)
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    print("Score: {}".format(score))

In [ ]:
# example for inferencing the environment.
agent = Agent(state_size=83, action_size=4, seed=0, QNetwork=QNetwork)
state_dict = torch.load('checkPoint.pth',map_location=torch.device('cpu'))
agent.qnetwork_local.load_state_dict(state_dict)
run(agent)